Markdown for importsMarkdown for imports

In [90]:
# Retail Transaction Product Category Classification

### Imports

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [18]:
data = pd.read_csv('retail_data.csv')

In [19]:
print (data.head())
print (data.columns)
print (data['Date'])

   Transaction_ID  Customer_ID  ... Ratings           products
0       8691788.0      37249.0  ...     5.0     Cycling shorts
1       2174773.0      69749.0  ...     4.0         Lenovo Tab
2       6679610.0      30192.0  ...     2.0   Sports equipment
3       7232460.0      62101.0  ...     4.0      Utility knife
4       4983775.0      27901.0  ...     1.0  Chocolate cookies

[5 rows x 30 columns]
Index(['Transaction_ID', 'Customer_ID', 'Name', 'Email', 'Phone', 'Address',
       'City', 'State', 'Zipcode', 'Country', 'Age', 'Gender', 'Income',
       'Customer_Segment', 'Date', 'Year', 'Month', 'Time', 'Total_Purchases',
       'Amount', 'Total_Amount', 'Product_Category', 'Product_Brand',
       'Product_Type', 'Feedback', 'Shipping_Method', 'Payment_Method',
       'Order_Status', 'Ratings', 'products'],
      dtype='object')
0          9/18/2023
1         12/31/2023
2          4/26/2023
3           05-08-23
4           01-10-24
             ...    
302005     1/20/2024
302006    12

## Data Cleaning

Remove unnecessary columns with specific user_transaction information such as Name, Email, Phone, etc. Similarly, remove the redundant columns: Year and Month as they are already represented in 'Date'.

In [20]:
data = data.drop(columns=['City', 'State', 'Zipcode', 'Country', 'Transaction_ID', 'Customer_ID', 'Address', 'Name', 'Email', 'Phone', 'Order_Status', 'Payment_Method', 'products', 'Year', 'Month', 'Time'])

### Data Exploration

In [21]:
unique_counts = data.nunique()

print("Number of unique values per attribute")
print(unique_counts)

Number of unique values per attribute
Age                     53
Gender                   2
Income                   3
Customer_Segment         3
Date                   366
Total_Purchases         10
Amount              299297
Total_Amount        299305
Product_Category         5
Product_Brand           18
Product_Type            33
Feedback                 4
Shipping_Method          3
Payment_Method           4
Order_Status             4
Ratings                  5
products               318
dtype: int64


In [22]:

for column in data.columns:
    unique_values = data[column].unique()
    
    print(f"Unique values in '{column}':")
    print(unique_values)
    print("-" * 50)

Unique values in 'Transaction_ID':
[8691788. 2174773. 6679610. ... 7157113. 4733214. 8201755.]
--------------------------------------------------
Unique values in 'Customer_ID':
[37249. 69749. 30192. ... 49840. 58286. 29845.]
--------------------------------------------------
Unique values in 'Name':
['Michelle Harrington' 'Kelsey Hill' 'Scott Jensen' ...
 'Caroline Anderson' 'Darlene Gomez' 'Kayla Stanley']
--------------------------------------------------
Unique values in 'Email':
['Ebony39@gmail.com' 'Mark36@gmail.com' 'Shane85@gmail.com' ...
 'Isaiah45@gmail.com' 'Karl9@gmail.com' 'Madeline91@gmail.com']
--------------------------------------------------
Unique values in 'Phone':
[1.41478680e+09 6.85289999e+09 8.36216045e+09 ... 1.11492579e+09
 5.69519790e+09 5.92317182e+09]
--------------------------------------------------
Unique values in 'Address':
['3959 Amanda Burgs' '82072 Dawn Centers' '4133 Young Canyon' ...
 '572 Wagner Forge' '197 Donna Lodge Apt. 673' '6026 Quinn Row S

In [23]:
for column in data.columns:
    value_counts = data[column].value_counts(normalize=True) * 100

    print(f"Percentage of unique values in '{column}':")
    print(value_counts)
    print("-" * 50)

Percentage of unique values in 'Transaction_ID':
Transaction_ID
6145934.0    0.001326
3823276.0    0.000994
9918529.0    0.000994
3859425.0    0.000994
7257551.0    0.000994
               ...   
4466731.0    0.000331
1234470.0    0.000331
7157113.0    0.000331
4733214.0    0.000331
2174773.0    0.000331
Name: proportion, Length: 294461, dtype: float64
--------------------------------------------------
Percentage of unique values in 'Customer_ID':
Customer_ID
99355.0    0.004309
48453.0    0.004309
90983.0    0.004309
49274.0    0.004309
60341.0    0.004309
             ...   
73258.0    0.000331
66041.0    0.000331
78480.0    0.000331
42671.0    0.000331
50893.0    0.000331
Name: proportion, Length: 86766, dtype: float64
--------------------------------------------------
Percentage of unique values in 'Name':
Name
Michael Smith        0.047741
Michael Johnson      0.036800
Michael Jones        0.033816
Christopher Smith    0.030833
Michael Williams     0.030170
                       

In [24]:
rows_with_nans = data[data.isna().any(axis=1)]

print("Rows with NaN values:")
print(rows_with_nans)
print("NUM ROWS", len(rows_with_nans))

print("NUM RECORDS", len(data))

Rows with NaN values:
         Age  Gender  Income  ... Feedback Shipping_Method  Ratings
109     65.0    Male     Low  ...      NaN        Same-Day      NaN
123     39.0    Male  Medium  ...      NaN        Standard      NaN
142     37.0    Male    High  ...  Average             NaN      2.0
174     50.0    Male    High  ...  Average        Standard      2.0
232      NaN  Female     Low  ...  Average        Same-Day      2.0
...      ...     ...     ...  ...      ...             ...      ...
301515  36.0  Female  Medium  ...      NaN         Express      NaN
301567  37.0    Male     Low  ...      Bad             NaN      1.0
301738  35.0    Male     NaN  ...      Bad        Standard      1.0
301875  44.0    Male    High  ...      Bad        Same-Day      1.0
301883  31.0    Male     NaN  ...     Good        Same-Day      4.0

[3497 rows x 14 columns]
NUM ROWS 3497
NUM RECORDS 302010


In [25]:
data = data.dropna()
data.head()
print("Num Remaining", len(data))

Num Remaining 298513


## Feature Engineering

In [26]:
item_to_category = {
    'Water': 'Drink',
    'Juice': 'Drink',
    'Soft Drink': 'Drink',
    'Coffee': 'Drink',
    'Chocolate': 'Food',
    'Snacks': 'Food',
    'Non-Fiction': 'Media',
    'Fiction': 'Media',
    'Thriller': 'Media',
    'Literature': 'Media',
    'Television': 'Media',
    'Mitsubishi AC': 'HVAC',
    'BlueStar AC': 'HVAC',
    'Fridge': 'Kitchen Equipment',
    'Kitchen': 'Kitchen Equipment',
    'T-Shirt': 'Upper Body Clothing',
    'Shirt': 'Upper Body Clothing',
    'Dress': 'Upper Body Clothing',
    'Jacket': 'Upper Body Clothing',
    'Shoes': 'Lower Body Clothing',
    'Jeans': 'Lower Body Clothing',
    'Shorts': 'Lower Body Clothing',
    'Decorations': 'Home Decor',
    'Furniture': 'Home Decor',
    'Lighting': 'Home Decor',
    'Bathroom': 'Home Decor',
    'Bedding': 'Home Decor',
    'Smartphone': 'Electronics',
    'Tablet': 'Electronics',
    'Headphones': 'Electronics',
    'Laptop': 'Electronics',
    'Children\'s': 'Children\'s',
    'Tools': 'Tools'
}

data['Product_Subcategory'] = data['Product_Type'].map(item_to_category)
last_8_columns = data.iloc[:, -8:]
print(last_8_columns.head())

data = data.drop(columns=['Product_Type'])

   Total_Amount Product_Category  ... Ratings  Product_Subcategory
0    324.086270         Clothing  ...     5.0  Lower Body Clothing
1    806.707815      Electronics  ...     4.0          Electronics
2   1063.432799            Books  ...     2.0           Children's
3   2466.854021       Home Decor  ...     4.0                Tools
4    248.553049          Grocery  ...     1.0                 Food

[5 rows x 8 columns]


Transform the 'Date' column into continuous feature value in column 'day_of_year'. Allows for data normalization and standardization of date format. 

In [27]:
data['Date'] = pd.to_datetime(data['Date'], errors='coerce', format=None)
data['day_of_year'] = data['Date'].dt.dayofyear
data = data.drop(columns=['Date'])

### Data Normalization

In [28]:
data_encoded = pd.get_dummies(data, columns=['Gender', 'Feedback', 'Product_Subcategory', 'Income', 'Product_Category', 'Product_Brand', 'Customer_Segment', 'Shipping_Method'], drop_first=True)
data_encoded.head()

,Age,Total_Purchases,Amount,Total_Amount,Ratings,day_of_year,Gender_Male,Feedback_Bad,Feedback_Excellent,Feedback_Good,Product_Subcategory_Drink,Product_Subcategory_Electronics,Product_Subcategory_Food,Product_Subcategory_HVAC,Product_Subcategory_Home Decor,Product_Subcategory_Kitchen Equipment,Product_Subcategory_Lower Body Clothing,Product_Subcategory_Media,Product_Subcategory_Tools,Product_Subcategory_Upper Body Clothing,Income_Low,Income_Medium,Product_Category_Clothing,Product_Category_Electronics,Product_Category_Grocery,Product_Category_Home Decor,Product_Brand_Apple,Product_Brand_Bed Bath & Beyond,Product_Brand_BlueStar,Product_Brand_Coca-Cola,Product_Brand_HarperCollins,Product_Brand_Home Depot,Product_Brand_IKEA,Product_Brand_Mitsubhisi,Product_Brand_Nestle,Product_Brand_Nike,Product_Brand_Penguin Books,Product_Brand_Pepsi,Product_Brand_Random House,Product_Brand_Samsung,Product_Brand_Sony,Product_Brand_Whirepool,Product_Brand_Zara,Customer_Segment_Premium,Customer_Segment_Regular,Shipping_Method_Same-Day,Shipping_Method_Standard
0,21.0,3.0,108.028757,324.086270,5.0,261.0,True,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,True,False
1,19.0,2.0,403.353907,806.707815,4.0,365.0,False,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,True
2,48.0,3.0,354.477600,1063.432799,2.0,116.0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False
3,56.0,7.0,352.407717,2466.854021,4.0,NaN,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True
4,22.0,2.0,124.276524,248.553049,1.0,NaN,True,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True


In [106]:
data_normalized = data_encoded.copy()
scaler = StandardScaler()

data_normalized[data_normalized.columns] = scaler.fit_transform(data_normalized[data_normalized.columns])

data_normalized.head()

,Age,Total_Purchases,Amount,Total_Amount,Ratings,day_of_year,Gender_Male,Feedback_Bad,Feedback_Excellent,Feedback_Good,Product_Subcategory_Drink,Product_Subcategory_Electronics,Product_Subcategory_Food,Product_Subcategory_HVAC,Product_Subcategory_Home Decor,Product_Subcategory_Kitchen Equipment,Product_Subcategory_Lower Body Clothing,Product_Subcategory_Media,Product_Subcategory_Tools,Product_Subcategory_Upper Body Clothing,Income_Low,Income_Medium,Product_Category_Clothing,Product_Category_Electronics,Product_Category_Grocery,Product_Category_Home Decor,Product_Brand_Apple,Product_Brand_Bed Bath & Beyond,Product_Brand_BlueStar,Product_Brand_Coca-Cola,Product_Brand_HarperCollins,Product_Brand_Home Depot,Product_Brand_IKEA,Product_Brand_Mitsubhisi,Product_Brand_Nestle,Product_Brand_Nike,Product_Brand_Penguin Books,Product_Brand_Pepsi,Product_Brand_Random House,Product_Brand_Samsung,Product_Brand_Sony,Product_Brand_Whirepool,Product_Brand_Zara,Customer_Segment_Premium,Customer_Segment_Regular,Shipping_Method_Same-Day,Shipping_Method_Standard
0,-0.963585,-0.822788,-1.040422,-0.924476,1.391299,0.688151,0.779929,-0.409361,1.413194,-0.678433,-0.47151,-0.405179,-0.202977,-0.086873,-0.40316,-0.217975,3.377047,-0.501776,-0.141702,-0.253569,1.461541,-0.871397,2.124313,-0.555716,-0.533335,-0.468663,-0.252324,-0.253494,-0.086873,-0.254741,-0.254652,-0.252865,-0.251776,-0.15105,-0.252685,3.954918,-0.252805,-0.334255,-0.252947,-0.254413,-0.254301,-0.159358,-0.254562,-0.520507,1.031382,1.377072,-0.679012
1,-1.096721,-1.171406,1.048464,-0.496891,0.634196,1.676687,-1.282168,-0.409361,1.413194,-0.678433,-0.47151,2.468045,-0.202977,-0.086873,-0.40316,-0.217975,-0.296117,-0.501776,-0.141702,-0.253569,1.461541,-0.871397,-0.470740,1.799482,-0.533335,-0.468663,-0.252324,-0.253494,-0.086873,-0.254741,-0.254652,-0.252865,-0.251776,-0.15105,-0.252685,-0.252850,-0.252805,-0.334255,-0.252947,3.930613,-0.254301,-0.159358,-0.254562,1.921203,-0.969573,-0.726178,1.472728
2,0.833762,-0.822788,0.702753,-0.269442,-0.880009,-0.690096,0.779929,-0.409361,-0.707617,-0.678433,-0.47151,-0.405179,-0.202977,-0.086873,-0.40316,-0.217975,-0.296117,-0.501776,-0.141702,-0.253569,1.461541,-0.871397,-0.470740,-0.555716,-0.533335,-0.468663,-0.252324,-0.253494,-0.086873,-0.254741,-0.254652,-0.252865,-0.251776,-0.15105,-0.252685,-0.252850,3.955622,-0.334255,-0.252947,-0.254413,-0.254301,-0.159358,-0.254562,-0.520507,1.031382,1.377072,-0.679012
3,1.366309,0.571682,0.688113,0.973937,0.634196,NaN,0.779929,-0.409361,1.413194,-0.678433,-0.47151,-0.405179,-0.202977,-0.086873,-0.40316,-0.217975,-0.296117,-0.501776,7.057059,-0.253569,-0.684209,-0.871397,-0.470740,-0.555716,-0.533335,2.133729,-0.252324,-0.253494,-0.086873,-0.254741,-0.254652,3.954683,-0.251776,-0.15105,-0.252685,-0.252850,-0.252805,-0.334255,-0.252947,-0.254413,-0.254301,-0.159358,-0.254562,1.921203,-0.969573,-0.726178,1.472728
4,-0.897016,-1.171406,-0.925499,-0.991396,-1.637112,NaN,0.779929,2.442834,-0.707617,-0.678433,-0.47151,-0.405179,4.926663,-0.086873,-0.40316,-0.217975,-0.296117,-0.501776,-0.141702,-0.253569,1.461541,-0.871397,-0.470740,-0.555716,1.874994,-0.468663,-0.252324,-0.253494,-0.086873,-0.254741,-0.254652,-0.252865,-0.251776,-0.15105,3.957501,-0.252850,-0.252805,-0.334255,-0.252947,-0.254413,-0.254301,-0.159358,-0.254562,1.921203,-0.969573,-0.726178,1.472728


In [34]:
traning_data, test_data = train_test_split(data_normalized, test_size=0.2, random_state=42)